<a href="https://colab.research.google.com/github/karthisha-13/EXPLAINABLE-AI_2303A52099_LAB/blob/main/ASSIGNMENT_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame({
    "Temperature": [30, 32, 29, 31, 33],
    "IceCreamsSold": [300, 330, 290, 310, 350]
})

X, y = df[["Temperature"]], df["IceCreamsSold"]
model = LinearRegression().fit(X, y)

baseline = y.mean()
df["Prediction"] = model.predict(X)
df["SHAP"] = df["Prediction"] - baseline
df["Baseline+SHAP"] = baseline + df["SHAP"]
df["Diff"] = df["Prediction"] - y
df["Over/Under"] = df["Diff"].apply(lambda v: "Over" if v > 0 else "Under" if v < 0 else "Exact")

print(df[["Temperature", "IceCreamsSold", "Prediction", "SHAP", "Baseline+SHAP", "Over/Under"]])

for i, r in df.iterrows():
    e = "increase" if r.SHAP > 0 else "decrease" if r.SHAP < 0 else "no change"
    print(f"{r.Temperature}°C → {e} of {abs(r.SHAP):.2f} from baseline {baseline:.2f}")


   Temperature  IceCreamsSold  Prediction  SHAP  Baseline+SHAP Over/Under
0           30            300       301.0 -15.0          301.0       Over
1           32            330       331.0  15.0          331.0       Over
2           29            290       286.0 -30.0          286.0      Under
3           31            310       316.0   0.0          316.0       Over
4           33            350       346.0  30.0          346.0      Under
30°C → decrease of 15.00 from baseline 316.00
32°C → increase of 15.00 from baseline 316.00
29°C → decrease of 30.00 from baseline 316.00
31°C → no change of 0.00 from baseline 316.00
33°C → increase of 30.00 from baseline 316.00


In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame({
    "Emails": [100, 150, 120, 80, 60],
    "Social": [5, 10, 8, 4, 2],
    "Enroll": [300, 500, 400, 250, 180]
})

X, y = df[["Emails", "Social"]], df["Enroll"]
m = LinearRegression().fit(X, y)

b0, b1, b2 = m.intercept_, *m.coef_
base = y.mean()
mx1, mx2 = df["Emails"].mean(), df["Social"].mean()

df["Pred"] = m.predict(X)
df["SHAP_E"] = b1 * (df["Emails"] - mx1)
df["SHAP_S"] = b2 * (df["Social"] - mx2)
df["Base+SHAP"] = base + df["SHAP_E"] + df["SHAP_S"]
df["Err"] = df["Pred"] - y
df["Over/Under"] = df["Err"].apply(lambda e: "Over" if e > 0 else "Under" if e < 0 else "Exact")

print(df[["Emails", "Social", "Enroll", "Pred", "SHAP_E", "SHAP_S", "Base+SHAP", "Over/Under"]])

for i, r in df.iterrows():
    print(f"Row{i+1}: Emails {r.SHAP_E:.2f}, Social {r.SHAP_S:.2f} → Pred {r.Pred:.2f}, Act {r.Enroll}, {r['Over/Under']}")


   Emails  Social  Enroll        Pred     SHAP_E     SHAP_S   Base+SHAP  \
0     100       5     300  304.812834  -3.090909 -18.096257  304.812834   
1     150      10     500  495.187166  74.181818  95.005348  495.187166   
2     120       8     400  403.582888  27.818182  49.764706  403.582888   
3      80       4     250  251.283422 -34.000000 -40.716578  251.283422   
4      60       2     180  175.133690 -64.909091 -85.957219  175.133690   

  Over/Under  
0       Over  
1      Under  
2       Over  
3       Over  
4      Under  
Row1: Emails -3.09, Social -18.10 → Pred 304.81, Act 300, Over
Row2: Emails 74.18, Social 95.01 → Pred 495.19, Act 500, Under
Row3: Emails 27.82, Social 49.76 → Pred 403.58, Act 400, Over
Row4: Emails -34.00, Social -40.72 → Pred 251.28, Act 250, Over
Row5: Emails -64.91, Social -85.96 → Pred 175.13, Act 180, Under


In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression().fit(X_train, y_train)
baseline = y_train.mean()
means = X_train.mean()

preds = model.predict(X_test)
shap_vals = (X_test - means) * model.coef_
shap_sum = shap_vals.sum(axis=1)

df = X_test.copy()
df["Actual"] = y_test
df["Pred"] = preds
df["Baseline+SHAP"] = baseline + shap_sum
df["Over/Under"] = df["Pred"] - df["Actual"]

print("Baseline:", baseline)
print("Coefficients:", dict(zip(X.columns, model.coef_)))
print(df.head())

for i, row in df.head(3).iterrows():
    print(f"\nPatient {i+1}: Pred={row.Pred:.1f}, Actual={row.Actual}, Error={row['Over/Under']:.1f}")
    for f in X.columns:
        val = shap_vals.loc[i, f]
        print(f"  {f}: {val:+.2f}")

Baseline: 153.73654390934846
Coefficients: {'age': np.float64(37.904021350074835), 'sex': np.float64(-241.96436231273972), 'bmi': np.float64(542.4287585162907), 'bp': np.float64(347.70384391385596), 's1': np.float64(-931.4888458835159), 's2': np.float64(518.0622769833373), 's3': np.float64(163.41998299131063), 's4': np.float64(275.3179015786486), 's5': np.float64(736.1988589046838), 's6': np.float64(48.67065743196508)}
          age       sex       bmi        bp        s1        s2        s3  \
287  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
211  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
72   0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
321  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
73   0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

           s4        s5        s6  Actual        Pred  Baseline+SHAP  \
287  0.034309  0.032432 -0.005220   219.0 

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import shap

df = pd.read_csv('/Experience_Salary.csv/')
X = df.drop(columns=['Salary'])
y = df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression().fit(X_train, y_train)

baseline = y_train.mean()
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

preds = model.predict(X_test)
verify = baseline + shap_values.values.sum(axis=1)

for i in range(len(X_test)):
    print(f"Record {i+1}")
    print(f"Predicted: {preds[i]:.2f}, Actual: {y_test.iloc[i]}, Baseline: {baseline:.2f}")
    for f, val in zip(X.columns, shap_values.values[i]):
        print(f"{f}: {val:+.2f}")
    print(f"Sum SHAP + Baseline: {verify[i]:.2f}")
    print()

Record 1
Predicted: 115790.21, Actual: 112635, Baseline: 74207.62
YearsExperience: +41582.59
Sum SHAP + Baseline: 115790.21

Record 2
Predicted: 71498.28, Actual: 67938, Baseline: 74207.62
YearsExperience: -2709.35
Sum SHAP + Baseline: 71498.28

Record 3
Predicted: 102596.87, Actual: 113812, Baseline: 74207.62
YearsExperience: +28389.24
Sum SHAP + Baseline: 102596.87

Record 4
Predicted: 75267.80, Actual: 83088, Baseline: 74207.62
YearsExperience: +1060.18
Sum SHAP + Baseline: 75267.80

Record 5
Predicted: 55477.79, Actual: 64445, Baseline: 74207.62
YearsExperience: -18729.83
Sum SHAP + Baseline: 55477.79

Record 6
Predicted: 60189.70, Actual: 57189, Baseline: 74207.62
YearsExperience: -14017.93
Sum SHAP + Baseline: 60189.70

